Import processed news articles and respective hourly prices for coin. Create columns of differences in the future using `.diff()` and `pct_change()`.

Column to create:
- 1 hour 
- 4 hour
- 12 hour
- 1 day

In [1]:
import datetime as date
import time
import pprint as pp

import pandas as pd

from talib.abstract import * 

#message notifier
from sendMessage import sendmessage as sm

#import coin list
from getCoinList import getCoinList as coin_list

#set coin list
coin_list = coin_list()

pd.set_option('display.max_columns', None)

In [2]:
cols = ['close', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'date_time']

In [3]:
for coin in coin_list:
    # get file name
    file = coin + '_OHLCV.csv'

    # read file from previous step
    df = pd.read_csv('./OHLCV/' + file,
                     usecols=cols,
                     index_col='date_time',
                     engine='c',
                     infer_datetime_format=True,
                     float_precision='round_trip')
    df = df[~df.index.duplicated()]

    df = df.sort_index()

    # rename cols for Abstract API
    # https://github.com/mrjbq7/ta-lib

    inputs = {
        'open': df['open'].values,
        'high': df['high'].values,
        'low': df['low'].values,
        'close': df['close'].values,
        'volume': df['volumefrom'].values
    }

    # uses 9 period close prices
    df['SMA_9_close'] = SMA(inputs, timeperiod=9)

    # uses 25 period close prices (default)
    df['SMA_25_close'] = SMA(inputs, timeperiod=25)

    # uses 50 period close prices
    df['SMA_50_close'] = SMA(inputs, timeperiod=50)

    # uses 100 period close prices
    df['SMA_100_close'] = SMA(inputs, timeperiod=100)

    # uses close prices (default)
    df['bbands_upper'], df['bbands_middle'], df['bbands_lower'] = BBANDS(inputs)

    # uses high, low, close (default)
    df['STOCH_slowk'], df['STOCH_slowD'] = STOCH(inputs) # uses high, low, close by default

    # uses high, low, open instead
    df['STOCH_slowk'], df['STOCH_slowD']  = STOCH(inputs)

    # ADX 
    df['ADX_14'] = ADX(inputs, timeperiod=14)

    # ATR
    df['ATR_14'] = ATR(inputs, timeperiod=14)

    # OBV
    df['OBV'] = OBV(inputs)

    # MACD
    df['macd'], df['macdsignal'], df['macdhist'] = MACD(inputs)

    #  MFI
    df['MFI'] = MFI(inputs)

    # RSI
    df['RSI'] = RSI(inputs)

    # MINMAX 365
    df['min_365'], df['max_365'] = MINMAX(inputs, timeperiod=365)

    # MINMAX 180
    df['min_180'], df['max_180'] = MINMAX(inputs, timeperiod=180)

    # MINMAX 90
    df['min_90'], df['max_90'] = MINMAX(inputs, timeperiod=90)

    # MINMAX 30
    df['min_30'], df['max_30'] = MINMAX(inputs, timeperiod=30)

    # MINMAX 14
    df['min_14'], df['max_14'] = MINMAX(inputs, timeperiod=14)

    # MINMAX 7
    df['min_7'], df['max_7'] = MINMAX(inputs, timeperiod=7)

    #HT_TRENDMODE
    df['HT_TRENDMODE']  = HT_TRENDMODE(inputs)
    
    # create diffs

    df['1_hr_diff'] = df['close'].diff(1)

    df['4_hr_diff'] = df['close'].diff(4)

    df['12_hr_diff'] = df['close'].diff(12)

    df['24_hr_diff'] = df['close'].diff(24)

    # create precent change

    df['1_hr_pct_change'] = df['close'].pct_change(1)

    df['4_hr_pct_change'] = df['close'].pct_change(4)

    df['12_hr_pct_change'] = df['close'].pct_change(12)

    df['24_hr_pct_change'] = df['close'].pct_change(24)

    # create shift in close

    df['close_1'] = df[['close']].shift(1)

    df['close_4'] = df[['close']].shift(4)

    df['close_12'] = df[['close']].shift(12)

    df['close_24'] = df[['close']].shift(24)

    # create isUp column

    df['isUp_1_hr'] = (df[['1_hr_pct_change']] > 0).astype(int)

    df['isUp_4_hr'] = (df[['4_hr_pct_change']] > 0).astype(int)

    df['isUp_12_hr'] = (df[['12_hr_pct_change']] > 0).astype(int)

    df['isUp_24_hr'] = (df[['24_hr_pct_change']] > 0).astype(int)

     # save a new file.
    newfile = 'OHLCV_TA_' + coin + '.csv'

    df.to_csv('./OHLCV_TA/' + newfile)
    print(coin + ' TA complete')
    
print('All Done')
sm('finished running priceTechnicalAnalysis.')

BTC TA complete
ETH TA complete
BCH TA complete
XRP TA complete
EOS TA complete
LTC TA complete
XLM TA complete
ADA TA complete
XMR TA complete
TRX TA complete
ZEC TA complete
NEO TA complete
DASH TA complete
MCO TA complete
USDT TA complete
SYS TA complete
All Done


In [22]:
df.tail()

,close,high,low,open,volumefrom,volumeto,SMA_9_close,SMA_25_close,SMA_50_close,SMA_100_close,bbands_upper,bbands_middle,bbands_lower,STOCH_slowk,STOCH_slowD,ADX_14,ATR_14,OBV,macd,macdsignal,macdhist,MFI,RSI,min_365,max_365,min_180,max_180,min_90,max_90,min_30,max_30,min_14,max_14,min_7,max_7,HT_TRENDMODE
date_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-07-09 13:00:00,0.1678,0.1678,0.1623,0.1633,913971.09,153364.35,0.162278,0.165084,0.167230,0.174250,0.168125,0.16284,0.157555,69.105691,44.173442,34.633380,0.002693,3.234950e+08,-0.001412,-0.001957,0.000545,63.277227,58.201982,0.1514,0.9227,0.1604,0.9227,0.1604,0.1929,0.1604,0.1718,0.1604,0.1678,0.1604,0.1678,1
2018-07-09 14:00:00,0.1667,0.1685,0.1637,0.1680,657767.02,109649.76,0.162900,0.164880,0.167108,0.173945,0.169461,0.16410,0.158739,83.827995,65.612042,33.053970,0.002843,3.228373e+08,-0.001042,-0.001774,0.000732,72.033084,54.792814,0.1514,0.9227,0.1604,0.9227,0.1604,0.1929,0.1604,0.1718,0.1604,0.1678,0.1604,0.1678,1
2018-07-09 15:00:00,0.1646,0.1666,0.1640,0.1666,390468.56,64271.12,0.163244,0.164652,0.166910,0.173651,0.169292,0.16478,0.160268,74.962865,75.965517,31.587375,0.002833,3.224468e+08,-0.000907,-0.001600,0.000693,72.600889,48.903534,0.1514,0.9227,0.1604,0.9227,0.1604,0.1929,0.1604,0.1718,0.1604,0.1678,0.1604,0.1678,1
2018-07-09 16:00:00,0.1631,0.1647,0.1614,0.1645,286780.65,46773.92,0.163389,0.164308,0.166714,0.173348,0.168844,0.16508,0.161316,49.962865,69.584575,29.644888,0.002866,3.221600e+08,-0.000911,-0.001463,0.000551,74.915971,45.169005,0.1514,0.9227,0.1604,0.9227,0.1604,0.1929,0.1604,0.1718,0.1604,0.1678,0.1612,0.1678,1
2018-07-09 17:00:00,0.1623,0.1640,0.1623,0.1628,121289.11,19818.64,0.163433,0.164008,0.166508,0.173054,0.169070,0.16490,0.160730,28.083800,51.003177,27.841150,0.002783,3.220387e+08,-0.000968,-0.001364,0.000396,71.577745,43.271090,0.1514,0.9227,0.1604,0.9227,0.1604,0.1900,0.1604,0.1718,0.1604,0.1678,0.1616,0.1678,1


In [212]:
import talib

# dict of functions by group
pp.pprint (talib.get_function_groups())

{'Cycle Indicators': ['HT_DCPERIOD',
                      'HT_DCPHASE',
                      'HT_PHASOR',
                      'HT_SINE',
                      'HT_TRENDMODE'],
 'Math Operators': ['ADD',
                    'DIV',
                    'MAX',
                    'MAXINDEX',
                    'MIN',
                    'MININDEX',
                    'MINMAX',
                    'MINMAXINDEX',
                    'MULT',
                    'SUB',
                    'SUM'],
 'Math Transform': ['ACOS',
                    'ASIN',
                    'ATAN',
                    'CEIL',
                    'COS',
                    'COSH',
                    'EXP',
                    'FLOOR',
                    'LN',
                    'LOG10',
                    'SIN',
                    'SINH',
                    'SQRT',
                    'TAN',
                    'TANH'],
 'Momentum Indicators': ['ADX',
                         'ADXR',
                  

In [ ]:
MULT()